In [1]:
%matplotlib inline

import time
import numpy as np
import progressbar_renamed as progressbar
import copy
import networkx as nx
from networkx.algorithms import bipartite
from operator import itemgetter


import import_ipynb
import QAOA_for_QLS as qaoa
import Side_Calc_QLS_bipartite as sc

importing Jupyter notebook from QAOA_for_QLS.ipynb
importing Jupyter notebook from Side_Calc_QLS.ipynb


In [2]:
def top_gains_populate_subset(la, subset_size, gains):
    if len(la) <= subset_size:
        return la
    return [
        x[0] for x in sorted(gains.items(), key=itemgetter(1))[-subset_size:]
    ]

In [9]:
def iteration_step(M, curr_solution, subset, lis1, lis2): #isoliert probiert, funktioniert!!

    n=len(subset)
    
    
    C = [0.0] * len(subset)
    subset=set(subset)
    la=set(list(lis1)+list(lis2))
    lis1=set(lis1)
    lis2=set(lis2)
    
    not_in_set=set(la)-subset
    ss1=lis1.intersection(subset)
    ss2=lis2.intersection(subset)
    nss1=lis1.intersection(not_in_set)
    nss2=lis2.intersection(not_in_set)
    
    lis1=sorted(list(lis1))
    lis2=sorted(list(lis2))
    la=lis1+lis2
    subset=sorted(list(ss1))+sorted(list(ss2))
    
    subset2global = dict((x, la.index(subset[x])) for x in range(0, n))

    indices1=[]
    indices2=[]
    for i in ss1:
        indices1.append(lis1.index(i))
        for j in nss2:
            C[subset.index(i)]+=M[lis1.index(i)][lis2.index(j)]*curr_solution[la.index(j)]
    for j in ss2:
        indices2.append(lis2.index(j))
        for i in nss1:
            C[subset.index(j)]+=M[lis1.index(i)][lis2.index(j)]*curr_solution[la.index(i)]
    
    #only keep indices from subset in M, transform M into quadratic Matrix for qaoa
    M=np.asarray(M)
    ixgrid=np.ix_(indices1,indices2)
    M=M[ixgrid]  
    Mq=[[0 for x in range(n)] for y in range(n)]
    for i in range(len(indices1)):
        for j in range(len(indices2)):
            Mq[i][j+len(indices1)]=M[i][j] #mal 2 wegen diagonal, oder nicht damit mit db etc zusammenpasst?
    
    result = qaoa.qaoa_basic(len(subset), Mq,C)
    optimized_subset=result.x
        
    for i in range(0, len(list(ss1))):
        curr_solution[subset2global[i]] = (optimized_subset[i])
    for j in range(len(list(ss1)), len(subset)):
        curr_solution[subset2global[j]] = (optimized_subset[j])
        
    return curr_solution
   

def QLS(M,lis1,lis2,size_of_iteration=12,stopping_criteria=3):
    
    la=lis1+lis2
    n=len(la)
    # random initial guess
    curr_solution = list(np.random.randint(2, size=n))
    
    curr_energy = sc.compute_energy(M, curr_solution)
    
    all_time_best_solution = curr_solution
    all_time_best_energy = curr_energy

    visited = set()
    it = 0
    it_stuck = 0
    all_energies = []
    
    while (n - len(visited)):
        it += 1
        if it_stuck > stopping_criteria: 
            break
        gains_list = []
        for v in progressbar.progressbar(la):
            gains_list.append(sc.compute_gain(M, curr_solution, la.index(v), True))
        #print(gains_list)
        gains = {la[v]: gain for gain, v in gains_list} 
       #print(gains)
        
        
        subset = top_gains_populate_subset(la, size_of_iteration, gains)
        
        cand_solution = iteration_step(M, copy.deepcopy(curr_solution), list(subset),lis1,lis2)
        cand_energy = sc.compute_energy(M, cand_solution)
        
        #print('it', it, 'cand_energy', cand_energy, 'curr_best',
              #all_time_best_energy)
        #return cand_energy, cand_solution
        
        all_energies.append({
            'it': it,
            'cand_energy': cand_energy,
            'curr_best': all_time_best_energy
        })
        if cand_energy > curr_energy:
            curr_energy = cand_energy
            curr_energy = cand_energy
            it_stuck = 0
        else:
            it_stuck += 1
        if curr_energy > all_time_best_energy:
            all_time_best_solution = curr_solution
            all_time_best_energy = curr_energy
        
        #break Bedingung?, vgl.: single_level_refinement
    return (all_time_best_energy, all_time_best_solution, it,
            all_energies)
        

In [10]:

M=[[1,1,1,0,0,0],[1,1,1,0,0,0],[1,1,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]]
M=np.array(M)
#cur_solution=np.random.randint(2, size=12)
#cur_solution=[1,1,1,1,1,1,1,1,1,1,1,1]
#cur_solution=[0,1,1,0,1,1,0,1,1,0,0,0,]
#vorher=curr_solution
#print(cur_solution)
#subset=(18,37,68,25,52)
lis1=(13,18,26,37,51,68)
lis2=(8,11,25,48,52,66)

QLS(M, lis1,lis2)

100% (12 of 12) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


it 1 cand_energy 12.0 curr_best 2.0


0